In [1]:
import pyift as ift
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx

%matplotlib notebook

/home/jordao/myenv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jordao/myenv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def show(img, dpi=125):
    fig, ax=plt.subplots(dpi=dpi)
    ax.imshow(img)

    
def drawGraph(G, Z, dpi=150):
    fig, ax=plt.subplots(dpi=dpi)
    coord = Z.GetData()
    coord[:,1] = coord[:, 1] * -1
    nx.draw_networkx(G, pos=coord, node_color=Z.GetGroups(), ax=ax, cmap = 'gist_rainbow',
                     node_size=50, alpha=0.5, linewidths=0.01, edge_color = 'grey', with_labels=False)
    
    
def imageOverlay(original, label):
    A = ift.Circular(1.0)
    tmp = ift.CopyImage(original)
    cmap = ift.CreateColorTable(ift.MaximumValue(label) + 1)
    ift.DrawLabels(tmp, label, cmap, A, True, 0.5)
    show(tmp.ToPlot())
    
    
def unsupTrainOPF(Z, kmax):
    mst = ift.CreateMST(Z)
    graph = ift.MSTtoKnnGraph(mst, kmax)
    ift.UnsupTrain(graph, ift.NormalizedCutPtr())
    return graph

In [3]:
# ATTENTION WITHOUT CORRECT PATHS IT WON'T WORK
# img = ift.ReadImageByExt("/home/jordao/Documents/grabcut/orig/000001_000007.ppm") # CHANGE THIS PATH 
# seeds = ift.ReadSeeds("/home/jordao/Documents/ift/demo/Classification/07_2.txt", img) # ALSO THIS ONE
img = ift.ReadImageByExt("/home/jordao/Documents/diversas/bears1.ppm") # CHANGE THIS PATH 
seeds = ift.ReadSeeds("/home/jordao/Documents/ift/demo/Classification/bears1.txt", img) # ALSO THIS ONE
n_suppxl = 30
A = ift.Circular(1.0)

In [ ]:
# suppxl = ift.Extract_ISF_MIX_ROOT_Superpixels(img, n_suppxl, 0.5, 12, 10, 2)
suppxl = ift.Extract_ISF_MIX_MEAN_Superpixels(img, n_suppxl, 0.5, 12, 10, 2)
border = ift.BorderImage(suppxl, False)
imageOverlay(img, border)

In [ ]:
feat = ift.ExtractImageFeatures(img, suppxl, A, False) # ATTENTION (True or False)
Z = ift.MImageToDataSet(feat, suppxl)
ift.SetStatus(Z, ift.IFT_TRAIN)
# ift.WriteDataSet(Z, "bears1.zip")

In [ ]:
ift.LabelDataSetFromSeeds(Z, seeds, suppxl)
# ift.WriteDataSet(Z, "bears1Seeds.zip")
labels = ift.DataSetToLabelImage(Z, suppxl, False, True)
imageOverlay(img, labels)

In [ ]:
knnGraph = unsupTrainOPF(Z, 5)
cluster = ift.DataSetClusterInformationToLabelImage(Z, suppxl, False)
imageOverlay(img, cluster)
# ift.WriteDataSet(Z, "bears1Cluster.zip")

In [ ]:
# Z2d = ift.DimReductionByTSNE(Z, 2, 40, 1000)
Z2d = ift.DimReductionByTSNE(Z, 2, 5, 1000)
plot = ift.Draw2DFeatureSpace(Z2d, ift.IFT_GROUP, ift.IFT_ALL)
show(plot.ToPlot(), dpi=200)

In [ ]:
G = nx.Graph(knnGraph.AsDict())
drawGraph(G, Z2d)

In [ ]:
cplGraph = ift.SemiSupTrain(Z)
ift.Classify(cplGraph, Z)
labels = ift.DataSetToLabelImage(Z, suppxl, False, False)
imageOverlay(img, labels)

In [ ]:
# Z2d = ift.DimReductionByTSNE(Z, 2, 40, 1000)
Z2d = ift.DimReductionByTSNE(Z, 2, 5, 1000)
plot = ift.Draw2DFeatureSpace(Z2d, ift.IFT_LABEL, ift.IFT_ALL)
show(plot.ToPlot(), dpi=200)

In [ ]:
cplGraph = ift.SemiSupTrain(Z)
ZTest = ift.CopyDataSet(Z, True)
ift.AddStatus(ZTest, ift.IFT_TEST)
ift.ClassifyWithCertaintyValues(cplGraph, ZTest)
objmap = ift.DataSetObjectMap(ZTest, suppxl, 255, -1)
show(objmap.ToPlot())

In [ ]:
mimg = ift.ImageToMImage(img, ift.LABNorm_CSPACE)
mask = ift.DynamicSetRootPolicy(mimg, A, seeds, 1, False)
# mask = ift.DynamicSetObjectPolicy(mimg, A, seeds, False)
imageOverlay(img, mask)

In [ ]:
# mimg = ift.ImageToMImage(img, ift.LABNorm_CSPACE)
# extSeeds = ift.LabeledSetFromSeedImage(labels, True)
# mask = ift.DynamicSetRootPolicy(mimg, A, extSeeds, 0, True)
# imageOverlay(img, mask)

In [ ]:
# ZSup = ift.ReadDataSet("/home/jordao/Documents/ift/demo/Classification/bears1Sup.zip")
# labelsSup = ift.DataSetToLabelImage(ZSup, suppxl, False, True)
# imageOverlay(img, labelsSup)

In [ ]:
# cplGraph = ift.SemiSupTrain(ZSup)
# ift.Classify(cplGraph, ZSup)
# labels = ift.DataSetToLabelImage(ZSup, suppxl, False, False)
# imageOverlay(img, labels)00

In [ ]:
# mimg = ift.ImageToMImage(img, ift.LABNorm_CSPACE)
# extSeeds = ift.LabeledSetFromSeedImage(labelsSup, True)
# mask = ift.DynamicSetRootPolicy(mimg, A, extSeeds, 0, True)
# # mask = ift.WaterCut(mimg, A, extSeeds, None)
# imageOverlay(img, mask)